<a href="https://colab.research.google.com/github/awadell1/MLOnAGooseberryBush/blob/master/src/LSTM_pretrainedW2V_noclean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation, Bidirectional
from keras.layers.embeddings import Embedding
from keras.preprocessing.text import text_to_word_sequence
from tqdm.notebook import tqdm
import pandas as pd
from tensorflow.keras.preprocessing.sequence import pad_sequences
SEED = 42 #setting the seed for reproducebility
AUTOTUNE = tf.data.experimental.AUTOTUNE #currently not entirely sure what this can do

doing cleaning locally due to concern about miss labeling 

In [ ]:
def read_glove_vecs(glove_file):
    with open(glove_file, 'r') as f:
        words = set()
        word_to_vec_map = {}
        for line in f:
            line = line.strip().split()
            curr_word = line[0]
            words.add(curr_word)
            word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)
        
        i = 1
        words_to_index = {}
        index_to_words = {}
        for w in sorted(words):
            words_to_index[w] = i
            index_to_words[i] = w
            i = i + 1
    return words_to_index, index_to_words, pd.DataFrame(word_to_vec_map)

In [ ]:
dict_pre_word_to_index100, dict_pre_index_to_word100, word_to_vec_map100 = read_glove_vecs('/content/drive/MyDrive/24787 Group Project  /glove.6B/glove.6B.100d.txt')

In [ ]:
dict_pre_word_to_index300, dict_pre_index_to_word300, word_to_vec_map300 = read_glove_vecs('/content/drive/MyDrive/24787 Group Project  /glove.6B/glove.6B.300d.txt')

In [ ]:
raw_df=pd.read_csv('/content/drive/MyDrive/24787 Group Project  /Data/winemag-data-130k-v2.csv',index_col=0)
raw_data=raw_df[['description']]
raw_label=raw_df[['points']]

In [ ]:
def review_cleanup(x):
  review_ls=text_to_word_sequence(x[0],filters='­­"…–â€˜‘’—!#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n\r“”')
  return review_ls

In [ ]:
tqdm.pandas()
raw_data['review_separated']=raw_data.progress_apply(lambda x: review_cleanup(x),axis=1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
I2W_df=pd.read_csv('/content/drive/My Drive/24787 Group Project  /Data/for_final_submission/Dictionary_new.csv')

I2W=I2W_df.to_dict(orient='dict')['Vocab']
I2W[0] = ''
W2I = {v: k for k, v in I2W.items()}

In [ ]:
def sentence_to_indices(data,w2ind):
    sentences_num=[]
    for row in tqdm(data):
        temp_mem=[w2ind[word] for word in row]
        sentences_num.append(temp_mem)
    return sentences_num

In [ ]:
sentences_num = sentence_to_indices(data=raw_data['review_separated'],w2ind=W2I)

In [ ]:
for seq in sentences_num[:2]:
  print(f"{seq} \n => \n {[I2W[i] for i in seq]}")

[15, 873, 202, 13, 1978, 3581, 1, 88, 105, 2, 16, 908, 1030, 928, 308, 14444, 48, 60, 1, 88, 421, 142, 360, 18] 
 => 
 ['aromas', 'include', 'tropical', 'fruit', 'broom', 'brimstone', 'and', 'dried', 'herb', 'the', 'palate', "isn't", 'overly', 'expressive', 'offering', 'unripened', 'apple', 'citrus', 'and', 'dried', 'sage', 'alongside', 'brisk', 'acidity']
[6, 7, 27, 1, 75, 3, 8, 17, 7, 99, 61, 123, 144, 78, 22, 28, 1064, 90, 5, 72, 30, 43, 49, 1, 3751, 5, 18, 24, 379, 701, 291, 12, 87, 684, 96, 439, 20, 340] 
 => 
 ['this', 'is', 'ripe', 'and', 'fruity', 'a', 'wine', 'that', 'is', 'smooth', 'while', 'still', 'structured', 'firm', 'tannins', 'are', 'filled', 'out', 'with', 'juicy', 'red', 'berry', 'fruits', 'and', 'freshened', 'with', 'acidity', "it's", 'already', 'drinkable', 'although', 'it', 'will', 'certainly', 'be', 'better', 'from', '2016']


In [ ]:
reviews_num = sentences_num
max_review_length = max([len(seq) for seq in reviews_num])
input_sequences = np.array(pad_sequences(reviews_num, maxlen=max_review_length, padding='pre'))

In [ ]:
for seq in input_sequences[:1]:
  print(f"{seq} \n => \n {[I2W[i] for i in seq]}")

[    0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0    15   873   202    13  1978  3581     1    88
   105     2    16   908  1030   928   308 14444    48    60     1    88
   421   142   360    18] 
 => 
 ['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',

In [ ]:
def pretrained_embedding_layer(word_to_vec_map, index_to_word, dict_pre):
    """
    Creates a Keras Embedding() layer
    
    Arguments:
    word_to_vec_map -- dictionary mapping words to their GloVe vector representation.
    word_to_index -- dictionary mapping from words to their indices in the vocabulary (400,001 words)

    Returns:
    embedding_layer -- pretrained layer Keras instance
    """
    
    vocab_len = len(index_to_word)+1                  # adding 1 to fit Keras embedding (requirement)
    emb_dim = word_to_vec_map.shape[0]      # define dimensionality of your word vectors (= 256)
    
    ### START CODE HERE ###
    # Initialize the embedding matrix as a numpy array of zeros of shape (vocab_len, dimensions of word vectors = emb_dim)
    emb_matrix = np.zeros((vocab_len,emb_dim))
    
    # Set each row "index" of the embedding matrix to be the word vector representation of the "index"th word of the vocabulary
    for index, word in tqdm(index_to_word.items()): #skip the first item "<padding>"
        word_check = dict_pre.get(word)
        if word_check is not None:
           emb_matrix[index,:] = word_to_vec_map[word].to_numpy().reshape(emb_dim,)


    # Define Keras embedding layer with the correct output/input sizes, make it trainable. Use Embedding(...). Make sure to set trainable=False. 
    embedding_layer = Embedding(vocab_len, emb_dim,trainable=True,mask_zero=True) 
    ##TODO:is this maskz_zero correct here?
    ### END CODE HERE ###

    # Build the embedding layer, it is required before setting the weights of the embedding layer. Do not modify the "None".
    embedding_layer.build((None,))
    
    # Set the weights of the embedding layer to the embedding matrix. Your layer is now pretrained.
    embedding_layer.set_weights([emb_matrix])
    
    return embedding_layer

In [ ]:
#I2W.pop(0)

embedding_layer = pretrained_embedding_layer(word_to_vec_map100, I2W, dict_pre_word_to_index100)
print("weights[0][1][3] =", embedding_layer.get_weights()[0][1][3])


weights[0][1][3] = -0.50638


In [ ]:
def WineReviewMachine_V2(input_shape, word_to_vec_map, index_to_word, num_class, dict_pre):
    """
    Function creating the Emojify-v2 model's graph.
    
    Arguments:
    input_shape -- shape of the input, usually (max_len,)
    word_to_vec_map -- dictionary mapping every word in a vocabulary into its 50-dimensional vector representation
    word_to_index -- dictionary mapping from words to their indices in the vocabulary (400,001 words)

    Returns:
    model -- a model instance in Keras
    """
    
    ### START CODE HERE ###
    # Define sentence_indices as the input of the graph, it should be of shape input_shape and dtype 'int32' (as it contains indices).
    sentence_indices = Input(shape = input_shape, dtype = 'int32') #the max length 
    
    # Create the embedding layer pretrained with GloVe Vectors (≈1 line)
    embedding_layer = pretrained_embedding_layer(word_to_vec_map, index_to_word, dict_pre)
    
    # Propagate sentence_indices through your embedding layer, you get back the embeddings
    embeddings = embedding_layer(sentence_indices)   
    # max_batch_size=1024
    # partitions = tf.range(max_batch_size)
    # num_partitions = max_batch_size
    # x_unpacked = tf.dynamic_partition(embeddings, partitions, num_partitions, name='dynamic_unstack')
    # #x_unpacked = tf.unstack(embeddings) # defaults to axis 0, returns a list of tensors
    # mini_batched_processed=[]
    # for x in x_unpacked:
    #    partitions = tf.range(136)
    #    num_partitions = 136
    #    t_ls = tf.dynamic_partition(x, partitions, num_partitions, name='dynamic_unstack') 
    #    average_embedding = tf.reduce_mean(x, 0)
    #    processed=[]
    #    for t in t_ls:
    #       if tf.math.count_nonzero(t).numpy()[0]==0:
    #          results_tensor=average_embedding
    #       else:
    #          results_tensor=t
    #       processed.append(result_tensor)
    #    batched_embeddings=tf.concat(processed,0)
    #    mini_batched_embeddings.append(batched_embeddings)
    # embeddings=tf.concat(mini_batched_embeddings,0)
    
    # Propagate the embeddings through an LSTM layer with 128-dimensional hidden state
    # Be careful, the returned output should be a batch of sequences.
    X = Bidirectional(LSTM(256, return_sequences=True))(embeddings)
    # Add dropout with a probability of 0.5
    X = Dropout(0.5)(X)
    # Propagate X trough another LSTM layer with 128-dimensional hidden state
    # Be careful, the returned output should be a single hidden state, not a batch of sequences.
    X = Bidirectional(LSTM(256, return_sequences=False))(X)
    # Add dropout with a probability of 0.5
    X = Dropout(0.5)(X)
    # Propagate X through a Dense layer with softmax activation to get back a batch of 5-dimensional vectors.
    X = Dense(num_class)(X)
    # Add a softmax activation
    X = Activation("softmax")(X)
    
    # Create Model instance which converts sentence_indices into X.
    model = Model(inputs=sentence_indices, outputs=X)
    
    ### END CODE HERE ###
    
    return model

In [ ]:
num_class=3
model = WineReviewMachine_V2((max_review_length,), word_to_vec_map100, I2W, num_class, dict_pre_word_to_index100)
model.summary()


Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 136)]             0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 136, 100)          3316800   
_________________________________________________________________
bidirectional (Bidirectional (None, 136, 512)          731136    
_________________________________________________________________
dropout (Dropout)            (None, 136, 512)          0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 512)               1574912   
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 3)               

In [ ]:
#for some reason the input_sequences have a shape of 129973
#while the total example number is 129971
#i will randomly drop 2 input sequences since the fraction is very low
targ_ind = np.arange(input_sequences.shape[0])
np.random.shuffle(targ_ind)
input_sequences=input_sequences[targ_ind[2:]]

In [ ]:
y_label=raw_label.values
y_label_3levels = []
for i,y0 in enumerate(y_label):
  temp = 0
  if y0 <87:
    temp = int(0)
  elif y0>=87 and y0<93:
     temp = int(1.0)
  elif y0>=93: temp = int(2.0)
  y_label_3levels.append(temp)

In [ ]:
indices = y_label_3levels
depth = max(indices)+1
y_label_one_hot=tf.one_hot(indices, depth)

In [ ]:
#because we have 2 inputs, targets and contexts, I dont know how to use train_test_split on this 
#I created a home grown train_test_spliter

#first, shuffle the indices
targ_ind = np.arange(input_sequences.shape[0])
np.random.shuffle(targ_ind)
#second, assign around 80% of the indices to train

train_ind = targ_ind[:-10000]
test_ind = targ_ind[-10000:]
#third, assign the general data to train and test datatset 
x_train_vec=[]
y_train_label=[]
for i in tqdm(train_ind):
   x_train_vec_temp = input_sequences[i]
   y_train_label_temp = y_label_one_hot[i]
   x_train_vec.append(x_train_vec_temp)
   y_train_label.append(y_train_label_temp)
x_test_vec=[]
y_test_label=[]
for i in tqdm(test_ind):
   x_test_vec_temp = input_sequences[i]
   y_test_label_temp = y_label_one_hot[i]
   x_test_vec.append(x_test_vec_temp)
   y_test_label.append(y_test_label_temp)

In [ ]:
BATCH_SIZE = 1024
BUFFER_SIZE = 10000
#this is some tensorflow data cleaning function
dataset_train = tf.data.Dataset.from_tensor_slices((x_train_vec, y_train_label))
dataset_train = dataset_train.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
#dataset_train = dataset_train.batch(BATCH_SIZE, drop_remainder=True)
BATCH_SIZE = 1024
BUFFER_SIZE = 10000
dataset_test = tf.data.Dataset.from_tensor_slices((x_test_vec, y_test_label))
dataset_test = dataset_test.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
#dataset_test = dataset_test.batch(BATCH_SIZE, drop_remainder=True)
print(dataset_train)
print(dataset_test)

<BatchDataset shapes: ((1024, 136), (1024, 3)), types: (tf.int32, tf.float32)>
<BatchDataset shapes: ((1024, 136), (1024, 3)), types: (tf.int32, tf.float32)>


In [ ]:
dataset_train = dataset_train.cache().prefetch(buffer_size=AUTOTUNE)
dataset_test = dataset_test.cache().prefetch(buffer_size=AUTOTUNE)
print(dataset_train)
print(dataset_test)

<PrefetchDataset shapes: ((1024, 136), (1024, 3)), types: (tf.int32, tf.float32)>
<PrefetchDataset shapes: ((1024, 136), (1024, 3)), types: (tf.int32, tf.float32)>


In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
#trained 9 epochs
model.fit(dataset_train,epochs=1,verbose=10,validation_data=dataset_test)

In [ ]:
def WineReviewMachine_V3(input_shape, word_to_vec_map, index_to_word,dict_pre):
    """
    Function creating the Emojify-v2 model's graph.
    
    Arguments:
    input_shape -- shape of the input, usually (max_len,)
    word_to_vec_map -- dictionary mapping every word in a vocabulary into its 50-dimensional vector representation
    word_to_index -- dictionary mapping from words to their indices in the vocabulary (400,001 words)

    Returns:
    model -- a model instance in Keras
    """
    
    ### START CODE HERE ###
    # Define sentence_indices as the input of the graph, it should be of shape input_shape and dtype 'int32' (as it contains indices).
    sentence_indices = Input(shape = input_shape, dtype = 'int32') #the max length 
    
    # Create the embedding layer pretrained with GloVe Vectors (≈1 line)
    embedding_layer = pretrained_embedding_layer(word_to_vec_map, index_to_word,dict_pre)
    
    # Propagate sentence_indices through your embedding layer, you get back the embeddings
    embeddings = embedding_layer(sentence_indices)   
    # max_batch_size=1024
    # partitions = tf.range(max_batch_size)
    # num_partitions = max_batch_size
    # x_unpacked = tf.dynamic_partition(embeddings, partitions, num_partitions, name='dynamic_unstack')
    # #x_unpacked = tf.unstack(embeddings) # defaults to axis 0, returns a list of tensors
    # mini_batched_processed=[]
    # for x in x_unpacked:
    #    partitions = tf.range(136)
    #    num_partitions = 136
    #    t_ls = tf.dynamic_partition(x, partitions, num_partitions, name='dynamic_unstack') 
    #    average_embedding = tf.reduce_mean(x, 0)
    #    processed=[]
    #    for t in t_ls:
    #       if tf.math.count_nonzero(t).numpy()[0]==0:
    #          results_tensor=average_embedding
    #       else:
    #          results_tensor=t
    #       processed.append(result_tensor)
    #    batched_embeddings=tf.concat(processed,0)
    #    mini_batched_embeddings.append(batched_embeddings)
    # embeddings=tf.concat(mini_batched_embeddings,0)
    
    # Propagate the embeddings through an LSTM layer with 128-dimensional hidden state
    # Be careful, the returned output should be a batch of sequences.
    X = Bidirectional(LSTM(256, return_sequences=True))(embeddings)
    # Add dropout with a probability of 0.5
    X = Dropout(0.5)(X)
    # Propagate X trough another LSTM layer with 128-dimensional hidden state
    # Be careful, the returned output should be a single hidden state, not a batch of sequences.
    X = Bidirectional(LSTM(256, return_sequences=False))(X)
    # Add dropout with a probability of 0.5
    X = Dropout(0.5)(X)
    # Propagate X through a Dense layer with softmax activation to get back a batch of 5-dimensional vectors.
    X = Dense(20)(X)
    X = Dense(1)(X)
    # Add a softmax activation
    X = Activation("linear")(X)
    
    # Create Model instance which converts sentence_indices into X.
    model = Model(inputs=sentence_indices, outputs=X)
    
    ### END CODE HERE ###
    
    return model

In [ ]:
y_test=raw_label.values-80
#because we have 2 inputs, targets and contexts, I dont know how to use train_test_split on this 
#I created a home grown train_test_spliter

#first, shuffle the indices
targ_ind = np.arange(input_sequences.shape[0])
np.random.shuffle(targ_ind)
#second, assign around 80% of the indices to train

train_ind = targ_ind[:-10000]
test_ind = targ_ind[-10000:]
#third, assign the general data to train and test datatset 
x_train_vec=[]
y_train_label=[]
for i in tqdm(train_ind):
   x_train_vec_temp = input_sequences[i]
   y_train_label_temp = y_test[i]
   x_train_vec.append(x_train_vec_temp)
   y_train_label.append(y_train_label_temp)
x_test_vec=[]
y_test_label=[]
for i in tqdm(test_ind):
   x_test_vec_temp = input_sequences[i]
   y_test_label_temp = y_test[i]
   x_test_vec.append(x_test_vec_temp)
   y_test_label.append(y_test_label_temp)

In [ ]:
model_linear100 = WineReviewMachine_V3((max_review_length,), word_to_vec_map100, I2W, dict_pre_word_to_index100)
model_linear100.summary()


Model: "functional_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 136)]             0         
_________________________________________________________________
embedding_5 (Embedding)      (None, 136, 100)          3316800   
_________________________________________________________________
bidirectional_6 (Bidirection (None, 136, 512)          731136    
_________________________________________________________________
dropout_6 (Dropout)          (None, 136, 512)          0         
_________________________________________________________________
bidirectional_7 (Bidirection (None, 512)               1574912   
_________________________________________________________________
dropout_7 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 20)              

In [ ]:
model_linear100.compile(loss=tf.keras.losses.MeanSquaredError(), optimizer='adam', metrics=[tf.keras.losses.MeanSquaredError()])
#trained 9 epochs
model_linear100.fit(dataset_train,epochs=20,verbose=1,validation_data=dataset_test)

Epoch 1/20
117/117 [==============================] - 61s 524ms/step - loss: 0.3292 - mean_squared_error: 0.3292 - val_loss: 0.2228 - val_mean_squared_error: 0.2228
Epoch 2/20
117/117 [==============================] - 58s 494ms/step - loss: 0.2255 - mean_squared_error: 0.2255 - val_loss: 0.2225 - val_mean_squared_error: 0.2225
Epoch 3/20
117/117 [==============================] - 58s 496ms/step - loss: 0.2247 - mean_squared_error: 0.2247 - val_loss: 0.2224 - val_mean_squared_error: 0.2224
Epoch 4/20
117/117 [==============================] - 58s 496ms/step - loss: 0.2242 - mean_squared_error: 0.2242 - val_loss: 0.2223 - val_mean_squared_error: 0.2223
Epoch 5/20
117/117 [==============================] - 58s 494ms/step - loss: 0.2238 - mean_squared_error: 0.2238 - val_loss: 0.2223 - val_mean_squared_error: 0.2223
Epoch 6/20
117/117 [==============================] - 58s 496ms/step - loss: 0.2235 - mean_squared_error: 0.2235 - val_loss: 0.2223 - val_mean_squared_error: 0.2223
Epoch 7/20

KeyboardInterrupt: ignored

start300D

In [ ]:
embedding_layer = pretrained_embedding_layer(word_to_vec_map300, I2W, dict_pre_word_to_index300)
print("weights[0][1][3] =", embedding_layer.get_weights()[0][1][3])


weights[0][1][3] = -0.38923


In [ ]:
y_label=raw_label.values
y_label_3levels = []
for i,y0 in enumerate(y_label):
  temp = 0
  if y0 <87:
    temp = int(0)
  elif y0>=87 and y0<93:
     temp = int(1.0)
  elif y0>=93: temp = int(2.0)
  y_label_3levels.append(temp)

In [ ]:
indices = y_label_3levels
depth = max(indices)+1
y_label_one_hot=tf.one_hot(indices, depth)

In [ ]:
#because we have 2 inputs, targets and contexts, I dont know how to use train_test_split on this 
#I created a home grown train_test_spliter

#first, shuffle the indices
targ_ind = np.arange(input_sequences.shape[0])
np.random.shuffle(targ_ind)
#second, assign around 80% of the indices to train

train_ind = targ_ind[:-10000]
test_ind = targ_ind[-10000:]
#third, assign the general data to train and test datatset 
x_train_vec=[]
y_train_label=[]
for i in tqdm(train_ind):
   x_train_vec_temp = input_sequences[i]
   y_train_label_temp = y_label_one_hot[i]
   x_train_vec.append(x_train_vec_temp)
   y_train_label.append(y_train_label_temp)
x_test_vec=[]
y_test_label=[]
for i in tqdm(test_ind):
   x_test_vec_temp = input_sequences[i]
   y_test_label_temp = y_label_one_hot[i]
   x_test_vec.append(x_test_vec_temp)
   y_test_label.append(y_test_label_temp)

In [ ]:
BATCH_SIZE = 1024
BUFFER_SIZE = 10000
#this is some tensorflow data cleaning function
dataset_train = tf.data.Dataset.from_tensor_slices((x_train_vec, y_train_label))
dataset_train = dataset_train.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
#dataset_train = dataset_train.batch(BATCH_SIZE, drop_remainder=True)
BATCH_SIZE = 1024
BUFFER_SIZE = 10000
dataset_test = tf.data.Dataset.from_tensor_slices((x_test_vec, y_test_label))
dataset_test = dataset_test.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
#dataset_test = dataset_test.batch(BATCH_SIZE, drop_remainder=True)
print(dataset_train)
print(dataset_test)

<BatchDataset shapes: ((1024, 136), (1024, 3)), types: (tf.int32, tf.float32)>
<BatchDataset shapes: ((1024, 136), (1024, 3)), types: (tf.int32, tf.float32)>


In [ ]:
dataset_train = dataset_train.cache().prefetch(buffer_size=AUTOTUNE)
dataset_test = dataset_test.cache().prefetch(buffer_size=AUTOTUNE)
print(dataset_train)
print(dataset_test)

<PrefetchDataset shapes: ((1024, 136), (1024, 3)), types: (tf.int32, tf.float32)>
<PrefetchDataset shapes: ((1024, 136), (1024, 3)), types: (tf.int32, tf.float32)>


In [ ]:
num_class=3
model300 = WineReviewMachine_V2((max_review_length,), word_to_vec_map300, I2W, num_class, dict_pre_word_to_index300)
model300.summary()


Model: "functional_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 136)]             0         
_________________________________________________________________
embedding_7 (Embedding)      (None, 136, 300)          9950400   
_________________________________________________________________
bidirectional_8 (Bidirection (None, 136, 512)          1140736   
_________________________________________________________________
dropout_8 (Dropout)          (None, 136, 512)          0         
_________________________________________________________________
bidirectional_9 (Bidirection (None, 512)               1574912   
_________________________________________________________________
dropout_9 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 3)               

In [ ]:
model300.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model300.fit(dataset_train,epochs=30,verbose=1,validation_data=dataset_test)

Epoch 1/30
117/117 [==============================] - 71s 606ms/step - loss: 0.8757 - accuracy: 0.6335 - val_loss: 0.8608 - val_accuracy: 0.6414
Epoch 2/30
117/117 [==============================] - 68s 580ms/step - loss: 0.8683 - accuracy: 0.6364 - val_loss: 0.8616 - val_accuracy: 0.6414
Epoch 3/30
117/117 [==============================] - 67s 573ms/step - loss: 0.8644 - accuracy: 0.6364 - val_loss: 0.8650 - val_accuracy: 0.6414
Epoch 4/30
117/117 [==============================] - 67s 572ms/step - loss: 0.8528 - accuracy: 0.6373 - val_loss: 0.8805 - val_accuracy: 0.6383
Epoch 5/30
117/117 [==============================] - 67s 573ms/step - loss: 0.8213 - accuracy: 0.6482 - val_loss: 0.9243 - val_accuracy: 0.6169
Epoch 6/30
117/117 [==============================] - 68s 577ms/step - loss: 0.7720 - accuracy: 0.6695 - val_loss: 0.9725 - val_accuracy: 0.6060
Epoch 7/30
117/117 [==============================] - 67s 576ms/step - loss: 0.7149 - accuracy: 0.6934 - val_loss: 1.0312 - val_ac

In [ ]:
y_test=raw_label.values-80
#because we have 2 inputs, targets and contexts, I dont know how to use train_test_split on this 
#I created a home grown train_test_spliter

#first, shuffle the indices
targ_ind = np.arange(input_sequences.shape[0])
np.random.shuffle(targ_ind)
#second, assign around 80% of the indices to train

train_ind = targ_ind[:-10000]
test_ind = targ_ind[-10000:]
#third, assign the general data to train and test datatset 
x_train_vec=[]
y_train_label=[]
for i in tqdm(train_ind):
   x_train_vec_temp = input_sequences[i]
   y_train_label_temp = y_test[i]
   x_train_vec.append(x_train_vec_temp)
   y_train_label.append(y_train_label_temp)
x_test_vec=[]
y_test_label=[]
for i in tqdm(test_ind):
   x_test_vec_temp = input_sequences[i]
   y_test_label_temp = y_test[i]
   x_test_vec.append(x_test_vec_temp)
   y_test_label.append(y_test_label_temp)

In [ ]:
model_linear300 = WineReviewMachine_V3((max_review_length,), word_to_vec_map300, I2W, dict_pre_word_to_index300)
model_linear300.summary()
model_linear300.compile(loss=tf.keras.losses.MeanSquaredError(), optimizer='adam', metrics=[tf.keras.losses.MeanSquaredError()])
model_linear300.fit(dataset_train,epochs=30,verbose=1,validation_data=dataset_test)